# Classifying Adverse Drug Events (ADEs)

In [24]:
import pandas as pd
from datasets import load_dataset
import re
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [25]:
# Load dataset 
dataset = load_dataset("SetFit/ade_corpus_v2_classification")
df = dataset['train'].to_pandas()
df.head()

Repo card metadata block was not found. Setting CardData to empty.


,text,label,label_text
0,"On cessation of the injections, the retrocorne...",0,Not-Related
1,Median patient age was 52 years.,0,Not-Related
2,A whole brain irradiation was performed for 37...,0,Not-Related
3,Complex biochemical syndrome of hypocalcemia a...,0,Not-Related
4,The fastidious organism grew only on buffered ...,0,Not-Related


### Text Preprocessing

In [26]:
# Download NLTK data files
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lucaspenney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lucaspenney/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lucaspenney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Uses Penn Treebank POS tags to map parts of speech for better lemmatization with WordNet
def _get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    if treebank_tag.startswith('V'):
        return wordnet.VERB
    if treebank_tag.startswith('N'):
        return wordnet.NOUN
    if treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN  


def preprocess_text(text):
    """Lowercase, normalize whitespace, tokenize, remove punctuation/digits-only tokens, and lemmatize (POS-aware)."""
    if not isinstance(text, str) or not text.strip():
        return ''
    # Lowercase and normalize whitespace
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove tokens that are only punctuation/digits
    tokens = [t for t in tokens if re.search(r'[a-z]', t)]
    if not tokens:
        return ''

    # POS tag for better lemmatization
    pos_tags = pos_tag(tokens)
    lemmatized = [
        lemmatizer.lemmatize(token, pos=_get_wordnet_pos(tag))
        for token, tag in pos_tags
    ]
    return ' '.join(lemmatized)


df['processed_text'] = df['text'].apply(preprocess_text)
df.head()

,text,label,label_text,processed_text
0,"On cessation of the injections, the retrocorne...",0,Not-Related,on cessation of the injection the retrocorneal...
1,Median patient age was 52 years.,0,Not-Related,median patient age be year
2,A whole brain irradiation was performed for 37...,0,Not-Related,a whole brain irradiation be perform for 37.5g...
3,Complex biochemical syndrome of hypocalcemia a...,0,Not-Related,complex biochemical syndrome of hypocalcemia a...
4,The fastidious organism grew only on buffered ...,0,Not-Related,the fastidious organism grow only on buffer ch...
